**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
%tensorflow_version 2.x
!pip install scikit-video
from tensorflow import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from tensorflow.keras.models import Sequential,model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
p = Path('/content/drive/My Drive/Colab Notebooks/DQN')

TensorFlow 2.x selected.
     |████████████████████████████████| 2.3MB 2.9MB/s 


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

In our example $\epsilon = 0.1$, if we are training the agent, then there are 90% of probabilities that the agent perform the learned action and 10% to perform a random action. 
<!-- Adding the randomness threshold $\epsilon$ will prevent over-fitting which is similar to the drop-out in the neural network training. -->
It controls the trade-off between exploitation (perform the learned action) and exploration (performing a random action).



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        path = p / (str(e) + '.mp4')
        skvideo.io.vwrite(path.resolve(), self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #??

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=31 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

`board` is the 2D table of reward: if the value of one point in `board` is $>0$, then it means that there is a cheese; if the value of one point in `board` is $<0$, then it means that there is poison.

`position` is to modelize the 2D table of environment: the (2-)marginals have value $-1$ which represents the wall while the center has values $0$ everywhere which represents the space where the rat can move.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, 5)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
    
        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.5/17.0. Average score (-4.5)
Win/lose count 8.0/13.0. Average score (-4.75)
Win/lose count 9.5/18.0. Average score (-6.0)
Win/lose count 8.5/10.0. Average score (-4.875)
Win/lose count 11.0/10.0. Average score (-3.7)
Win/lose count 4.0/4.0. Average score (-3.0833333333333335)
Win/lose count 11.5/5.0. Average score (-1.7142857142857142)
Win/lose count 11.0/10.0. Average score (-1.375)
Win/lose count 12.5/13.0. Average score (-1.2777777777777777)
Win/lose count 11.0/15.0. Average score (-1.55)
Win/lose count 9.0/9.0. Average score (-1.4090909090909092)
Final score: -1.4090909090909092


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}






1. the $Q$-function when $T=\infty$: for all policy $\pi$,
\begin{align*}
Q^{\pi}(s,a) & =E[\sum_{t=0}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
& = r(s, a) \ + E[\sum_{t=1}^\infty\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
&= r(s, a) \ + \gamma E \left[ E[\sum_{t=1}^\infty\gamma^{t-1}r(s_{t},a_{t})|s_{1},a_{1}] \Bigg| s_{0}=s,a_{0}=a \right] \tag{1} \\
&= r(s, a) \ + \gamma E \left[ E[\sum_{t'=0}^\infty\gamma^{t'}r(s_{t'},a_{t'})|s'_{0},a'_{0}] \Bigg| s_{0}=s,a_{0}=a \right] \\
&= r(s, a) \ + \gamma E \left[ Q^{\pi} \Bigg| s_{0}=s,a_{0}=a \right] \\
&= E_{(s',a')\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)} \left[r(s, a) \ + \gamma Q^{\pi}(s', a')  \right] \tag{*}
\end{align*}
(1) is from a conditional expectation proprietary, see Proposition 11.3.2 in *Intégration, Probabilités et Processus Aléatoires*, Le Gall 2006 for more details. Note that (*) $E_{(s',a')\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)} \left[r(s, a) \ + \gamma Q(s', a')  \right]$ is a _wrong_ expression adapted to the question form since $a'$ only depends on policy $\pi$ and the current state $s$ _(and mathematically, $\mathbb{P}(.|s_{0}=s,a_{0}=a)$ is a conditional distribution over state space which means $(s',a')\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)$ does not make any sense)_. 
So it is in fact $E_{s'\sim \mathbb{P}(.|s_{0}=s,a_{0}=a), \ a' \sim \pi(.|s_0 = s)}\left[r(s, a) \ + \gamma Q(s', a')  \right]$.

2. the optimal Q function is:
\begin{align*}
Q^*(s,a)&=\max_{\pi}Q^\pi(s,a) \\
&= \max_{\pi}E_{s'\sim \mathbb{P}(.|s_{0}=s,a_{0}=a), \ a' \sim \pi(.|s_0 = s)} \left[r(s, a) \ + \gamma Q^{\pi}(s', a')  \right] \\
&= E_{s'\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)} \left[r(s, a) \ + \gamma \max_{\substack{\pi \\ a' \sim \pi(.|s_0 = s)}} Q^{\pi}(s', a')  \right]\\
&= E_{s'\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)} \left[r(s, a) \ + \gamma \max_{\substack{ a' }} Q^*(s', a')  \right]\tag{2}
\end{align*}
(2): No matter $\pi$ is stochastic or determinist, $a'\mapsto Q(s', a') $ is a determinit function and $\pi(.|s_0 = s)$ includes determinist decision rule of $a'$. So $$\max_{\substack{\pi \\ a' \sim \pi(.|s_0 = s)}} Q^{\pi}(s', a') = \max_{\substack{ a' }} Q^*(s', a') \quad \forall s'$$

3. If the Q-function is parametrized by $\theta$, then we want to minimized the (square loss in expectation) distance between $Q(s, a, \theta)$ and $r(s,a)+\gamma\max_{a'}Q(s',a',\theta), \ s'\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)$. So the loss we minimized is formally:
\begin{equation*}
\mathcal{L}(\theta)=E_{s'\sim \mathbb{P}(.|s_{0}=s,a_{0}=a)} \left[ \Vert r(s,a) +\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2} \right]
\end{equation*}
If $\mathcal{L}(\theta) = 0$, then it means that our parametrized $Q(.,., \theta)$ is the theoretical optimal Q-function.


---
**_Remark_**: in 1. and 2., we don't distinguish the real state $s$ and the observed state $\phi(s)$. However, we have no access to the real state $s$, so we only have one $r(s, a)$ if we choose action $a$ at each time and we don't know $r(s, a')$ for a different action $a'$.

In the implementation of Question 7, this makes getting the real values of `q_target` in the form of `[q_target(a_1), q_target(a_2), q_target(a_3), q_target(a_4)]` is impossible because we cannot get back to the past to know the reward of an action we didn't choose. So we need to change the dimension of `q_target` (1 instead of 4) and add `input_actions` (combining with `input_states` as the `inputs` of neural network) to perform a sound optimization in 3. (It is exactly the _Deep Q-learning with Experience Replay_ algorithm given in the lecture 4, P39 of slides).

---

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        Q_values = []
        for a in range(4):
            input_ = np.zeros(5*5*self.n_state + 1)
            input_[:-1] = s.ravel()
            input_[-1]  = a
            Q_values.append(self.model.predict(input_[None, :])[0,0])
        return np.argmax(Q_values)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # I changed the dimension of input and output, see the remark in Question 5 for explication of necessity
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        input_actions = np.zeros(self.batch_size)
        # target_q = np.zeros((self.batch_size, 4))
        target_q = np.zeros(self.batch_size)
        # inputs = np.zeros((self.batch_size, 5,5,self.n_state, ?))

        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            input_actions[i] = a_
            if game_over_:
                target_q[i] = r_
            else:
                target_q[i] = r_
                Q_values = []
                for a in range(4):
                    input_ = np.zeros(5*5*self.n_state + 1)
                    input_[:-1] = n_s_.ravel()
                    input_[-1]  = a
                    Q_values.append(self.model.predict(input_[None, :])[0,0])
                target_q[i] += self.discount * max(Q_values)
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        inputs = np.zeros((self.batch_size, 5*5*self.n_state + 1))
        inputs[:, :5*5*self.n_state] = input_states.reshape((self.batch_size, 5*5*self.n_state))
        inputs[:, -1] = input_actions
        l = self.model.train_on_batch(inputs, target_q)
        # print(l)
        # l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)

        name_model = p / name_model
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
        name_weights= str(p/name_weights)
        self.model.save_weights(name_weights, overwrite=True)
        
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        name_model = p / name_model
        
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))

        name_weights= str(p/name_weights)
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(4096, input_dim=5*5*self.n_state + 1, activation='relu'))
        # model.add(BatchNormalization())
        model.add((Dense(4, activation='sigmoid')))
        model.add((Dense(1)))
        
        model.compile(SGD(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/021 | Loss 0.0220 | Win/lose count 1.5/3.0 (-1.5)
Epoch 001/021 | Loss 0.0113 | Win/lose count 2.5/5.0 (-2.5)
Epoch 002/021 | Loss 0.0148 | Win/lose count 2.0/6.0 (-4.0)
Epoch 003/021 | Loss 0.0135 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/021 | Loss 0.0071 | Win/lose count 2.0/1.0 (1.0)
Epoch 005/021 | Loss 0.0020 | Win/lose count 2.0/2.0 (0.0)
Epoch 006/021 | Loss 0.0102 | Win/lose count 1.5/2.0 (-0.5)
Epoch 007/021 | Loss 0.0084 | Win/lose count 7.0/1.0 (6.0)
Epoch 008/021 | Loss 0.0048 | Win/lose count 3.0/5.0 (-2.0)
Epoch 009/021 | Loss 0.0145 | Win/lose count 5.0/5.0 (0.0)
Epoch 010/021 | Loss 0.0063 | Win/lose count 4.0/2.0 (2.0)
Epoch 011/021 | Loss 0.1897 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/021 | Loss 0.0188 | Win/lose count 2.0/3.0 (-1.0)
Epoch 013/021 | Loss 0.0096 | Win/lose count 3.5/4.0 (-0.5)
Epoch 014/021 | Loss 0.0094 | Win/lose count 2.0/3.0 (-1.0)
Epoch 015/021 | Loss 0.1700 | Win/lose count 2.0/0 (2.0)
Epoch 016/021 | Loss 0.0052 | Win/lose count 3.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
from tensorflow.keras.layers import Input, Reshape, Flatten
from tensorflow.keras.models import Model
# from tensorflow.keras.backend import slice as k_slice
from tensorflow.keras.backend import concatenate

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        state_action_input = Input(shape=(5*5*self.n_state + 1,))
        x = Reshape((5,5,self.n_state))(state_action_input[:, :-1])
        y = state_action_input[:, -1:]

        x = Conv2D(self.n_state, 2, data_format="channels_last", activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(data_format="channels_last")(x)
        x = Conv2D(4, 2, data_format="channels_last", activation='relu')(x)
        x = Flatten()(x)
        z = concatenate([x,y])
        pred = Dense(1)(z)

        model = Model(state_action_input, pred)

        model.compile(SGD(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/021 | Loss 0.0132 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/021 | Loss 0.0157 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/021 | Loss 0.0276 | Win/lose count 2.0/3.0 (-1.0)
Epoch 003/021 | Loss 0.0700 | Win/lose count 3.0/3.0 (0.0)
Epoch 004/021 | Loss 0.0083 | Win/lose count 2.0/3.0 (-1.0)
Epoch 005/021 | Loss 0.0089 | Win/lose count 2.0/5.0 (-3.0)
Epoch 006/021 | Loss 0.0893 | Win/lose count 2.0/1.0 (1.0)
Epoch 007/021 | Loss 0.0204 | Win/lose count 2.5/5.0 (-2.5)
Epoch 008/021 | Loss 0.0282 | Win/lose count 2.5/1.0 (1.5)
Epoch 009/021 | Loss 0.0704 | Win/lose count 0.5/2.0 (-1.5)
Epoch 010/021 | Loss 0.0317 | Win/lose count 2.0/7.0 (-5.0)
Epoch 011/021 | Loss 0.0014 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/021 | Loss 0.0077 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/021 | Loss 0.0017 | Win/lose count 2.0/3.0 (-1.0)
Epoch 014/021 | Loss 0.0598 | Win/lose count 7.0/7.0 (0.0)
Epoch 015/021 | Loss 0.0008 | Win/lose count 3.0/4.0 (-1.0)
Epoch 016/021 | Loss 0.0090 | Win/lose count 1

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.5/6.0. Average score (-2.5)
Win/lose count 3.0/2.0. Average score (-0.75)
Win/lose count 2.0/4.0. Average score (-1.1666666666666667)
Win/lose count 2.0/3.0. Average score (-1.125)
Win/lose count 5.0/4.0. Average score (-0.7)
Win/lose count 6.0/3.0. Average score (-0.08333333333333333)
Win/lose count 5.5/7.0. Average score (-0.2857142857142857)
Win/lose count 0.5/4.0. Average score (-0.6875)
Win/lose count 2.0/1.0. Average score (-0.5)
Win/lose count 3.0/1.0. Average score (-0.25)
Win/lose count 4.5/1.0. Average score (0.09090909090909091)
Final score: 0.09090909090909091
Test of the FC
Win/lose count 3.5/3.0. Average score (0.5)
Win/lose count 4.0/4.0. Average score (0.25)
Win/lose count 3.0/1.0. Average score (0.8333333333333334)
Win/lose count 5.5/5.0. Average score (0.75)
Win/lose count 10.5/5.0. Average score (1.7)
Win/lose count 8.5/1.0. Average score (2.6666666666666665)
Win/lose count 3.0/2.0. Average score (2.4285714285714284)
Win/lose count 4.

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

Issues are: 


1.   Agents do not explore enough the game (board)
2.   Agents get stuck in the corner

By increasing the temperature (with more cheese and poison), final scores are increased and the agent explore more space in the game.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    epsilon_ = agent.epsilon
    agent.set_epsilon(1)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initiall state
        state = env.reset()
        epsilon = max(epsilon_, agent.epsilon * 0.9)
        agent.set_epsilon(epsilon)
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            

            

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{:.4f} ({:.4f}) | epsilon {:.4f}"
              .format(e, epoch, loss, win, lose, win-lose, agent.epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')



class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        path = p / (str(e) + '.mp4')
        skvideo.io.vwrite(path.resolve(), self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #??

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        #changes
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        reward = reward + self.board[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
  

In [0]:
# Training
# I trained the NN(s) continuously by saving and loading them
# I prefer to use fc-NN rather than cnn-NN because of its better performance


# CNN
# env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
# agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)

# agent.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
# train_explore(agent, env, epochs_train, prefix='cnn_train_explore')

# HTML(display_videos('cnn_train_explore30.mp4'))

# FC

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128, n_state=3)

agent.load(name_weights='fc_train_exploremodel.h5',name_model='fc_train_exploremodel.json')

train_explore(agent, env, epochs_train, prefix='fc_train_explore')
HTML(display_videos('fc_train_explore30.mp4'))

Epoch 000/031 | Loss 0.0903 | Win/lose count 7.0/24.8000 (-17.8000) | epsilon 0.1000
Epoch 001/031 | Loss 0.0854 | Win/lose count 12.0/21.8000 (-9.8000) | epsilon 0.1000
Epoch 002/031 | Loss 0.1270 | Win/lose count 4.0/20.4000 (-16.4000) | epsilon 0.1000
Epoch 003/031 | Loss 0.0313 | Win/lose count 4.5/19.9000 (-15.4000) | epsilon 0.1000
Epoch 004/031 | Loss 0.0747 | Win/lose count 5.5/18.5000 (-13.0000) | epsilon 0.1000
Epoch 005/031 | Loss 0.0271 | Win/lose count 6.0/19.5000 (-13.5000) | epsilon 0.1000


In [0]:
# CNN Evaluation
# env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
# agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
# agent.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')

# test(agent,env,epochs_test,prefix='cnn_test_explore')
# HTML(display_videos('cnn_test_explore10.mp4'))

# FC Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 128,n_state=3)
agent.load(name_weights='fc_train_exploremodel.h5',name_model='fc_train_exploremodel.json')

test(agent,env,epochs_test,prefix='fc_test_explore')
HTML(display_videos('fc_test_explore10.mp4'))

Win/lose count 8.0/4.0. Average score (4.0)
Win/lose count 5.5/1.0. Average score (4.25)
Win/lose count 2.5/5.0. Average score (2.0)
Win/lose count 5.5/3.0. Average score (2.125)
Win/lose count 8.5/6.0. Average score (2.2)
Win/lose count 10.0/5.0. Average score (2.6666666666666665)
Win/lose count 8.0/1.0. Average score (3.2857142857142856)
Win/lose count 10.0/0. Average score (4.125)
Win/lose count 9.0/1.0. Average score (4.555555555555555)
Win/lose count 4.5/1.0. Average score (4.45)
Win/lose count 7.0/4.0. Average score (4.318181818181818)
Final score: 4.318181818181818


In [0]:
# CNN Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
agent.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')

test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 3.0/5.0. Average score (-2.0)
Win/lose count 1.0/2.0. Average score (-1.5)
Win/lose count 1.5/6.0. Average score (-2.5)
Win/lose count 3.0/2.0. Average score (-1.625)
Win/lose count 2.5/5.0. Average score (-1.8)
Win/lose count 1.5/3.0. Average score (-1.75)
Win/lose count 5.0/4.0. Average score (-1.3571428571428572)
Win/lose count 2.5/1.0. Average score (-1.0)
Win/lose count 1.0/5.0. Average score (-1.3333333333333333)
Win/lose count 2.5/3.0. Average score (-1.25)
Win/lose count 1.5/3.0. Average score (-1.2727272727272727)
Final score: -1.2727272727272727


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***